# Workshop on PBHs - Friends of Friends Meeting, 2022
## Session 2 - Magnetic Field Generation from PBH distributions (Araya - Rubio)

(Main Reference: Araya et. al., https://arxiv.org/pdf/2012.09585.pdf)

# Needed packages

In [1]:
# From Joaquín Sureda's code

from cosmo import cosmology as cosmo
from constants import G,c
from Mass_Functions_FCT import dndM_brk_Ms as FCT 
from Mass_Functions_HC import dndM_brk_Ms as HC

# Integration module (by M. San Martín)

from special_integral import special_integral

#From Python

import numpy as np 
import matplotlib.pyplot as plt
import scipy.special as special
from scipy.interpolate import InterpolatedUnivariateSpline
from scipy import integrate
from scipy.integrate import quadrature
from tqdm import tqdm_notebook as tqdm

#Define some special functions

def Si(x):
    return special.sici(x)[0]

def Ci(x):
    return special.sici(x)[1]

## Fundamental constants and units

In [2]:
#Units used: [L] = Mpc ; [M] = Msun ; [T] = seg ; [Carga] = C

#Equivalences

#Conversion factor        
#---------------------------------------------
m_to_Mpc = 3.24078e-23       #m to Mpc
cm_to_Mpc = 3.24078e-25      #cm to Mpc
kg_to_Msun = 5.0279e-31      #kg to Msun
g_to_Msun = 5.0279e-34       #g to Msun
Tesla_to_Gauss = 1e4         #Tesla to Gauss

# 1 Gauss = 10^{-4} kg * Coulomb^{-1} * seg^{-1}

Gauss_to_OurUnits = 1e-4 * kg_to_Msun     # Msun * Coulomb^{-1} * seg^{-1}

#Fundamental constants (IS)

I_epsilon_0 = 8.8541878176e-12         # Coulomb^2 Kg^{-1} * metro^{-3} * seg^2
I_mu_0 = 1.25663706e-6                 # metro * kg * Coulomb^{-2}
I_h_bar = 1.0545718e-34                # Kg * metro^2 * seg^{-1}
I_c = 299792458                        # metro * seg^{-1}
I_e = 1.60217656535e-19                # Coulomb

#Magnetic charge

I_q_m = 2. * np.pi * I_epsilon_0 * I_h_bar * (I_c)**2. * (I_e)**(-1.)    # Coulomb * metro * seg^{-1}

#------------------------------------------------------------------------------------------------------------
# Magnitude                                                          Unit
#------------------------------------------------------------------------------------------------------------
epsilon_0 = I_epsilon_0 * (kg_to_Msun)**(-1) * (m_to_Mpc)**(-3.)  # Coulomb^2 * Msun^{-1} * Mpc^{-3} * seg^2
mu_0 = I_mu_0 * m_to_Mpc * kg_to_Msun                             # Mpc * Msun * Coulomb^{-2}
h_bar = I_h_bar * kg_to_Msun * (m_to_Mpc)**2.                     # Msum * Mpc^2 * seg^{-1}
c = I_c * m_to_Mpc                                                # Mpc * seg^{-1}
e = I_e                                                           # Coulomb
q_m = 2. * np.pi * epsilon_0 * h_bar * c**2. * e**(-1.)           # Coulomb * Mpc * seg^{-1}

# Initialize the cosmology 

Here we get back to Joaquin's "cosmo" class and initialize the cosmology we shall use in order to compute the mass functions.

We choose and fix the mass function parameters, namely $M_*$, $n_b$ and $n_s$

Also, we set $z=20$ and compute the corresponding scale factor, that shall be used later on.

In [ ]:
c1 = cosmo()

# Mass function parameters

Mstar1 = 1.39e2
nb_FCT1 = 1.5
nb_HC = 3.0
ns = 0.9649

h_factor = 0.6736
z = 20.
a = 1./(1. + z) #Scale factor

#Horizon radius in comoving coordinates

R_horizon = c1.horizon(a, iscomoving=True)

# Mass functions and characteristic distance

## Plotting the mass functions

In [ ]:
#Mass ranges

#Evaporated mass at z=20 (Thanks Juan!)
ini_LogMass = -20. + np.log10(2.041402)

final_LogMass_FCT1_Mstar1 = 3.3472791597546196
final_LogMass_HC_Mstar1   = 3.934821069027584

LogMass_FCT1_Mstar1 = np.linspace(ini_LogMass,final_LogMass_FCT1_Mstar1,500)
LogMass_HC_Mstar1   = np.linspace(ini_LogMass,final_LogMass_HC_Mstar1,500)

#Mass functions in FCT and HC scenarios

Mass_func_FCT1_Mstar1 = np.vectorize(FCT)(LogMass_FCT1_Mstar1,a,c1,M_star=Mstar1,nb=nb_FCT1)
Mass_func_HC_Mstar1   = np.vectorize(HC)(LogMass_HC_Mstar1,a,c1,M_star=Mstar1,nb=nb_HC)

def graph_function_FCT1_Mstar1(u):
    return FCT(u,a,c1,M_star=Mstar1,nb=nb_FCT1)

def graph_function_HC_Mstar1(u):
    return HC(u,a,c1,M_star=Mstar1,nb=nb_HC)

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=12)     # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=10)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title

fig = plt.figure(figsize=(8, 5), dpi=400)
ax = fig.add_subplot(111)
ax.semilogy(LogMass_FCT1_Mstar1,graph_function_FCT1_Mstar1(LogMass_FCT1_Mstar1), color='r', linestyle='-', label=r'FCT, $n_b$ = 1.5')
ax.semilogy(LogMass_HC_Mstar1,graph_function_HC_Mstar1(LogMass_HC_Mstar1), color='b', linestyle='-.', label=r'HC, $n_b$ = 3.0')
ax.set_ylabel(r'$\frac{dn}{dM}(M)\; [M_\odot^{-1} Mpc^{-3}]$')
ax.set_xlabel(r'$\log_{10}\left(M/M_\odot\right)$')
plt.ylim([10.**(-10.), 10.**(46.)])
plt.grid()
ax.legend()
#plt.savefig('Mass-Functions.png')
plt.show()

# Characteristic distance between two equal-mass PBHs

## Integrating a function in log domain

Recall the characteristic distance between PBHs of mass $M$ is such that

$$
    \frac{1}{(d(M))^3}=\int_{M}^{\infty}{\frac{dn}{dM}}\simeq\int_{M}^{M1ph}{\frac{dn}{dM}}
$$

We need to compute

$$
\int_{M_1}^{M_2}{f(M)dM}
$$

but knowing $u\mapsto f(M(u))$, where $u:=\log(M)$. Then,

$$
\int_{M_1}^{M_2}{f(M)dM}=\int_{u_1}^{u_2}{f(M(u))10^{u}du}
$$

In [ ]:
# MASS FUNCTIONS (WITH LOG JACOBIAN INCLUDED, FOR LOG INTEGRATION)

def Mfunction_FCT1_Mstar1(u):
    return FCT(u,a,c1,M_star=Mstar1,nb=nb_FCT1)*np.log(10.)*np.float_power(10.,u)

def Mfunction_HC_Mstar1(u):
    return HC(u,a,c1,M_star=Mstar1,nb=nb_HC)*np.log(10.)*np.float_power(10.,u)

#Characteristic Distances

def ch_distance_FCT1_Mstar1(u):
    if u < final_LogMass_FCT1_Mstar1:
        arg = quadrature(Mfunction_FCT1_Mstar1, u, final_LogMass_FCT1_Mstar1, tol=10000, rtol=10000, maxiter=500)[0]
        return (arg)**(-1./3.)
    else:
        return R_horizon
    
def ch_distance_HC_Mstar1(u):
    if u < final_LogMass_HC_Mstar1:
        arg = quadrature(Mfunction_HC_Mstar1, u, final_LogMass_HC_Mstar1, tol=10000, rtol=10000, maxiter=500)[0]
        return (arg)**(-1./3.)
    else:
        return R_horizon
    
#Vectorize them

d_FCT1_Mstar1 = np.vectorize(ch_distance_FCT1_Mstar1)
d_HC_Mstar1   = np.vectorize(ch_distance_HC_Mstar1)

SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=12)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=8)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
fig = plt.figure(figsize=(8, 5), dpi=400)
ax = fig.add_subplot(111)
ax.semilogy(LogMass_FCT1_Mstar1,d_FCT1_Mstar1(LogMass_FCT1_Mstar1), color='r', linestyle='-', label=r'FCT, $n_b$ = 1.5')
ax.semilogy(LogMass_HC_Mstar1,d_HC_Mstar1(LogMass_HC_Mstar1), color='b', linestyle='-.',label=r'HC, $n_b$ = 3.0')
ax.set_xlabel(r'$\log_{10}(M/M_\odot)$')
ax.set_ylabel(r'$d(M)\,[Mpc]$')
#ax.set_title("Characteristic Distance Functions")
plt.grid()
ax.legend()
#plt.savefig('charact-distance.png')
plt.show()

# Matter Power Spectra ($z=20$)

Data taken from Planck collaboration, CMB Boltzmann code, CAMB and Poisson noise contribution 
(for details, see Padilla et. al, https://arxiv.org/pdf/2010.06470.pdf) (Thanks Nelson!)

Then, we interpolate the data within the interested frequency range. From the wavelength range of interest, $[\lambda_{min},\lambda_{max}]$, we define the corresponding frequency range $[k_{min},k_{max}]$, where

$$
    k_{max}=\frac{2\pi}{k_{min}}, \qquad k_{min}=\frac{2\pi}{k_{max}}
$$

All the computations in frequency domain shall be done within the above range.

In [ ]:
# PLANCK DATA

#Frequency ranges

DataPlanck_FCT1_Mstar1 = open('Planck-FCT1-Mstar1.dat', 'r')
LogK_FCT1_Mstar1       = [float(a.split()[0]) for a in DataPlanck_FCT1_Mstar1]

DataPlanck_HC_Mstar1   = open('Planck-HC-Mstar1.dat', 'r')
LogK_HC_Mstar1         = [float(a.split()[0]) for a in DataPlanck_HC_Mstar1]

# PLANCK
DataPlanck_FCT1_Mstar1 = open('Planck-FCT1-Mstar1.dat', 'r')
Planck_FCT1_Mstar1     = [10.**(float(a.split()[1])) for a in DataPlanck_FCT1_Mstar1]

DataPlanck_HC_Mstar1   = open('Planck-HC-Mstar1.dat', 'r')
Planck_HC_Mstar1       = [10.**(float(a.split()[1])) for a in DataPlanck_HC_Mstar1]

# POISSON
# We compute Log_10(P_Poisson(k))/h^3/Mpc^3=log10(10**(-#14)+10**(2*#13))+#12 (con #1 la de los k's)

DataPoisson_FCT1_Mstar1 = open('Pk20_FCT_2_1.5.dat', 'r')
Poisson_FCT1_Mstar1     = [(10.**((-1.)*float(a.split()[13])) + 10.**(2.*float(a.split()[12])))*(10.**(float(a.split()[11]))) for a in DataPoisson_FCT1_Mstar1]

DataPoisson_HC_Mstar1   = open('Pk20_HC_2_3.0.dat', 'r')
Poisson_HC_Mstar1       = [((10.**(- float(a.split()[13])) + 10.**(2. * float(a.split()[12])))*10.**(float(a.split()[11]))) for a in DataPoisson_HC_Mstar1]

# Interpolated spectra

k_bins = 1001

#log-log interpolation

Log_Planck_FCT1_Mstar1 = [np.log10(float(a)) for a in Planck_FCT1_Mstar1]
Log_Planck_HC_Mstar1   = [np.log10(float(a)) for a in Planck_HC_Mstar1]

Log_Poisson_FCT1_Mstar1 = [np.log10(float(a)) for a in Poisson_FCT1_Mstar1]
Log_Poisson_HC_Mstar1   = [np.log10(float(a)) for a in Poisson_HC_Mstar1]

#Frequency domain in log scale

LogK_Int_FCT1_Mstar1 = np.linspace(min(LogK_FCT1_Mstar1), max(LogK_FCT1_Mstar1), num=k_bins)
LogK_Int_HC_Mstar1   = np.linspace(min(LogK_HC_Mstar1), max(LogK_HC_Mstar1), num=k_bins)


# Spline interpolation for Planck data; linear interpolation for Poisson data

SPlanck_FCT1_Mstar1 = InterpolatedUnivariateSpline(LogK_FCT1_Mstar1, Log_Planck_FCT1_Mstar1)(LogK_Int_FCT1_Mstar1)
SPlanck_HC_Mstar1   = InterpolatedUnivariateSpline(LogK_HC_Mstar1, Log_Planck_HC_Mstar1)(LogK_Int_HC_Mstar1)
LPoisson_FCT1_Mstar1 = np.interp(LogK_Int_FCT1_Mstar1, LogK_FCT1_Mstar1, Log_Poisson_FCT1_Mstar1)
LPoisson_HC_Mstar1   = np.interp(LogK_Int_HC_Mstar1, LogK_HC_Mstar1, Log_Poisson_HC_Mstar1)


SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12
plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=12)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=8)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
fig = plt.figure(figsize=(8, 5), dpi=400)
ax = fig.add_subplot(111)
plt.plot(LogK_Int_FCT1_Mstar1, SPlanck_FCT1_Mstar1, label=r'$Spline-Planck-FCT1-M1$')
plt.plot(LogK_Int_HC_Mstar1, SPlanck_HC_Mstar1, label=r'$Spline-Planck-HC-M1$')
plt.plot(LogK_Int_FCT1_Mstar1, LPoisson_FCT1_Mstar1, label=r'$Linear-Poisson-FCT1-M1$')
plt.plot(LogK_Int_HC_Mstar1, LPoisson_HC_Mstar1, label=r'$Linear-Poisson-HC-M1$')
ax.set_xlabel(r'$\log_{10}{\left(k/h/Mpc\right)}$')
ax.set_ylabel(r'$P_{Planck}\;\;and\;\; P_{Poisson}\;\;[h^3Mpc^{-3}]$')
plt.grid()
ax.legend()
plt.show()
plt.close()

## Matter Spectra

In [ ]:
Planck_10to_FCT1_Mstar1 = [10.**(float(a)) for a in SPlanck_FCT1_Mstar1]
Planck_10to_HC_Mstar1   = [10.**(float(a)) for a in SPlanck_HC_Mstar1]

Poisson_10to_FCT1_Mstar1 = [10.**(float(a)) for a in LPoisson_FCT1_Mstar1]
Poisson_10to_HC_Mstar1   = [10.**(float(a)) for a in LPoisson_HC_Mstar1]

Psum_FCT1_Mstar1 = []
Psum_HC_Mstar1   = []

for i in range(k_bins):
    Psum_FCT1_Mstar1.append(Planck_10to_FCT1_Mstar1[i] + Poisson_10to_FCT1_Mstar1[i])
    Psum_HC_Mstar1.append(Planck_10to_HC_Mstar1[i] + Poisson_10to_HC_Mstar1[i])
    
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=12)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=9)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
fig = plt.figure(figsize=(8, 5), dpi=400)
ax = fig.add_subplot(111)
plt.semilogy(LogK_Int_FCT1_Mstar1, Psum_FCT1_Mstar1, color='r', linestyle='-', label=r'FCT $n_b = 1.5$, $M_* = 1.39\times 10^2\,M_\odot$')
plt.semilogy(LogK_Int_HC_Mstar1, Psum_HC_Mstar1, color='b', linestyle='-.', label=r'HC $n_b = 3.0$, $M_* = 1.39\times 10^2\,M_\odot$')
ax.set_xlabel(r'$\log_{10}{\left(k/h/Mpc\right)}$')
ax.set_ylabel(r'$P_{matter}(k)\;\;[h^3Mpc^{-3}]$')
plt.grid()
ax.legend()
#plt.savefig('MPS-extended-z20.png')
plt.show()
plt.close()

# Magnetic field generation from Biermann's battery

Biermann's battery predicts a mostly dipolar magnetic field, namely

$$
    B(M,r)\sim B_{*}(M)\left(\frac{r_{*}}{r}\right)^3
$$

Following the results by Safarzadeh (2018), we choose $r_*\simeq 4 r_{isco}$ and

$$
    B_{*}(M)\approx 10^{-2}\left[\frac{\text{Gauss}}{s}\right]\left(\frac{M}{5.0\, M_\odot}\right)^{-9/4}\left(\frac{G M}{r_*^3}\right)^{-1/2}.
$$

We also assume that all the PBHs have maximum rotation speed. With this reference value, let's estimate the contribution to the magnetic field generated by each PBH, at redshift $z$, as

$$
B(M,|\vec{x}^{\prime}-\vec{x}|, z) = \frac{B_{*}(M)}{a(z)^3}\left( \frac{r_*}{|\vec{x}^{\prime}-\vec{x}|}\right)^3
$$

where $a(z)$ is the cosmological scale factor at redshift $z$, and the interval $|\vec{x}^{\prime}-\vec{x}|$ is measured in comoving coordinates.

Then, the magnetic field strenght produced by the whole population is

$$
\delta B_i (\vec{x}, z) = \int_{\mathbb{R}^{3}} \mathcal{C}F_{B}\left(\left\vert \vec{x}^{\prime}-\vec{x}\right\vert ,z\right)  \delta(
\vec{x}^{\prime}, z)  d^{3}\vec{x}^{\prime},
$$

where

$$
    F_{B}(r,z)=\int_{0}^{\infty}\frac{dn}{dM}\,B(M,r, z)\,S\left(\frac{r}{d(M)}\right)\,dM,
$$

The magnetic power spectrum can be computed as

$$
    P_B(k)=|\widehat{F_{B}}(k, z)|^{2}\;P_{\Lambda\scriptsize{\mbox{CDM}}}(\vec{k}, z)
$$

where

$$
    \widehat{F_{B}}(\vec{k}, z) \simeq \int_{M_i}^{M_f}{\frac{dn}{dM} \,B(M,r, z)\,\widehat{G}_i(\vec{k},M)\,\mbox{d}M},
$$

In [35]:
# Compute the r_isco

def r_isco(u):
    M = np.float_power(10.,u)
    x = 1. 
    z1 = 1. + (1. - x**2.)**(1./3.) * ((1. + x)**(1./3.) + (1. - x)**(1./3.))
    z2 = (3. * x**2. + z1**2.)**(1./2.)
    corr = 3. + z2 - ((3.-z1)*(3.+z1+2.*z2))**(1./2.)
    risco = (G*M/c**2.) * corr 
    
    return risco

# Magnetic field generated by 1PBH set up

def B_Biermann_4_r_isco(u):

    risco = r_isco(u)
    M = np.float_power(10.,u)
    dBdt = 0.01 * (M/5.)**(-9./4.) #G/s
    omega = (G*M/(4.*risco)**3.)**(1./2.)
    tau = 1./omega 

    return tau*dBdt

# Integrands of |F_Bhat|^2

#Set the orientation parameter C to the largest possible

C = 1.

def integrand_Biermann_FCT1_Mstar1(LogMass, w):

    k = np.float_power(10.,w)
    M = np.float_power(10.,LogMass)
    dndM = Mfunction_FCT1_Mstar1(LogMass)
    B_isco = B_Biermann_4_r_isco(LogMass)
    risco = r_isco(LogMass)
    I_tita = 4. * np.pi * (2./(k * d_FCT1_Mstar1(LogMass)) * np.sin(k * d_FCT1_Mstar1(LogMass) / 2.) - Ci(k * d_FCT1_Mstar1(LogMass) / 2.))

    return C * dndM * B_isco * (4.*risco)**3. / (a**3.) * I_tita

def integrand_Biermann_HC_Mstar1(LogMass, w):

    k = np.float_power(10.,w)
    M = np.float_power(10.,LogMass)
    dndM = Mfunction_HC_Mstar1(LogMass)
    B_isco = B_Biermann_4_r_isco(LogMass)
    risco = r_isco(LogMass)
    I_tita = 4. * np.pi * (2./(k * d_HC_Mstar1(LogMass)) * np.sin(k * d_HC_Mstar1(LogMass) / 2.) - Ci(k * d_HC_Mstar1(LogMass) / 2.))

    return C * dndM * B_isco * (4.*risco)**3. / (a**3.) * I_tita

# F_Bhat Biermann

def F_B_hat_Biermann_FCT1_Mstar1(w):
    return quadrature(integrand_Biermann_FCT1_Mstar1, ini_LogMass, final_LogMass_FCT1_Mstar1, args=(w), tol=10000, rtol=10000, maxiter=500)[0]

def F_B_hat_Biermann_HC_Mstar1(w):
    return quadrature(integrand_Biermann_HC_Mstar1, ini_LogMass, final_LogMass_HC_Mstar1, args=(w), tol=10000, rtol=10000, maxiter=500)[0]

#Vectorize them

function_F_B_hat_Biermann_FCT1_Mstar1 = np.vectorize(F_B_hat_Biermann_FCT1_Mstar1)
function_F_B_hat_Biermann_HC_Mstar1   = np.vectorize(F_B_hat_Biermann_HC_Mstar1)

SqFBhat_Biermann_FCT1_Mstar1 = [(F_B_hat_Biermann_FCT1_Mstar1(a))**2. for a in LogK_Int_FCT1_Mstar1]
SqFBhat_Biermann_HC_Mstar1   = [(F_B_hat_Biermann_HC_Mstar1(a))**2. for a in LogK_Int_HC_Mstar1]

# 'Normalized' version

MagSp_Biermann_FCT1_Mstar1 = []
MagSp_Biermann_HC_Mstar1   = []

for i in range(k_bins):
    MagSp_Biermann_FCT1_Mstar1.append((10.**(3.*LogK_Int_FCT1_Mstar1[i]))/(2.*np.pi**2.)*SqFBhat_Biermann_FCT1_Mstar1[i]*Psum_FCT1_Mstar1[i])
    MagSp_Biermann_HC_Mstar1.append((10.**(3.*LogK_Int_HC_Mstar1[i]))/(2.*np.pi**2.)*SqFBhat_Biermann_HC_Mstar1[i]*Psum_HC_Mstar1[i])

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=12)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=8)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
fig = plt.figure(figsize=(8, 5), dpi=400)
ax = fig.add_subplot(111)
plt.semilogy(LogK_Int_FCT1_Mstar1, MagSp_Biermann_FCT1_Mstar1, color='r', linestyle='-', label=r'FCT, $n_b = 1.5$, $M_* = 1.39\times 10^2\,M_\odot$')
plt.semilogy(LogK_Int_HC_Mstar1, MagSp_Biermann_HC_Mstar1, color='b', linestyle='-', label=r'HC, $n_b = 3.0$, $M_* = 1.39\times 10^2\,M_\odot$')
ax.set_xlabel(r'$\log_{10}{\left(k/h/Mpc\right)}$')
ax.set_ylabel(r'$k^3\,P_B(k)/Mpc^3/h^3$')
plt.grid()
ax.legend()
#plt.savefig('Biermann-PowerSpectrum.png')
plt.show()
plt.close()

With the above power spectrum, we compute the average magnetic field of the whole PBH population, smoothed on a certain scale $\lambda$, namely

$$
    \left\langle B^2_{\lambda}\right\rangle  = \frac{1}{2\pi^2}\int_0^{\infty}{P_B(k)\,k^2\,e^{-\lambda^2\,k^2}\,\mbox{d}k}
$$

at $z = 20$ and at a scale $\lambda\sim 1\,\mbox{kpc}$.

Finally, the average magnetic field can be approximated as

$$
    B \simeq \sqrt{\left\langle B^2_{\lambda}\right\rangle}
$$

In [ ]:
# Magnetic Average

L_jacob_FCT1_Mstar1 = [(np.log(10.) * 10.**(3.*u)) for u in LogK_Int_FCT1_Mstar1]
L_jacob_HC_Mstar1   = [(np.log(10.) * 10.**(3.*u)) for u in LogK_Int_HC_Mstar1]

lambda_par = 0.002 #Mpc

WF_FCT1_Mstar1 = [(1. / (2. * np.pi**2.)) * (np.exp(- lambda_par**2. * 10.**(2.*u))) for u in LogK_Int_FCT1_Mstar1]
WF_HC_Mstar1   = [(1. / (2. * np.pi**2.)) * (np.exp(- lambda_par**2. * 10.**(2.*u))) for u in LogK_Int_HC_Mstar1]

product_Bier_FCT1_Mstar1 = []
product_Bier_HC_Mstar1   = []

for i in range(k_bins):
    product_Bier_FCT1_Mstar1.append(WF_FCT1_Mstar1[i]*SqFBhat_Biermann_FCT1_Mstar1[i]*Psum_FCT1_Mstar1[i]*L_jacob_FCT1_Mstar1[i])
    product_Bier_HC_Mstar1.append(WF_HC_Mstar1[i]*SqFBhat_Biermann_HC_Mstar1[i]*Psum_HC_Mstar1[i]*L_jacob_HC_Mstar1[i])

int_product_Bier_FCT1_Mstar1 = integrate.simps(product_Bier_FCT1_Mstar1, LogK_Int_FCT1_Mstar1)
int_product_Bier_HC_Mstar1   = integrate.simps(product_Bier_HC_Mstar1, LogK_Int_HC_Mstar1)

Average_B_Biermann_FCT1_Mstar1 = (int_product_Bier_FCT1_Mstar1)**(1./2.)
Average_B_Biermann_HC_Mstar1   = (int_product_Bier_HC_Mstar1)**(1./2.)

print("-----------------------------------------------------")
print("B_Biermann_FCT1_Mstar1 = ", Average_B_Biermann_FCT1_Mstar1)
print("B_Biermann_HC_Mstar1   = ", Average_B_Biermann_HC_Mstar1)

Comments:

- We have assumed that every PBH have an accretion disk around and is able to drive the corresponding Biermann induction mechanism.
- From MHD considerations, it is unlikely for PBHs of masses lower than $10^{-22}\,M_{\odot}$ to form the required accretion disk, as the expected disk will be too faint, and thus not dense enough to be approximated by a fluid.
- We also assumed that maximal PBH spin.
- The Biermann battery seems not capable of producing the required seed field at $z=20$ in order to account, after evolution, for the present day magnetic fields in voids.

___________________________________________________________________

# Activity: PMFs from monopole accretion 

- Assume the existence of magnetic monopoles in the Universe
- Assume they can be accreted by the population of PBHs considered before.

We wonder: how much possible could be that the primordial magnetic field intensity needed to seed the current measurments have been generated from the accretion of magnetic monopoles?

Recall that the magnetic field at distance $r$ away from a PBH of mass $M$ is

$$
    B(M,r) = \frac{K\,\sqrt{M}}{r^{2}},
$$
where the orientation of the field is in the radial direction (with equal probability of pointing either towards the PBH or away from it). In comoving coordinates,
$$
    B_{\tiny{\text{Mald}}}(M, |\vec{x}^{\prime}-\vec{x}|, z) = \frac{K\,\sqrt{M}}{a(z)^2\,|\vec{x}^{\prime}-\vec{x}|^2},
$$
where $a(z)$ is the cosmological scale factor at redshift z.

The constant $K$ is given by

$$
    K = \frac{\mu_{0}}{4\pi} q_{m}\, \sqrt{\frac{n_{\text{mon}}}{\rho_{\tiny{\mbox{PBH}}}}}\,,
$$
where $n_{\text{mon}}$ is the average number density of magnetic monopoles in
the universe and $\rho_{\tiny{\mbox{PBH}}}$ is the average cosmic mass density of
PBHs. 



In order to see this:

Part 1: fixing  the constant $K$
--

1. Compute the average magnetic field generated by the accretion of monopoles onto the PBH population, assuming $K=1$.
2. For each one of the cosmological scenarios taken into account for the Biermann battery mechanism, estimate the minimum value of $K$ such that the the average magnetic field generated by the PBHs from monopole accretion is equal or greater than order $10^{-30}$ G at $z=20$.
3. Get back to the computation of the magnetic power spectra for this generation mechanism, with the $K$ values obtained before.

In [44]:
# Set K factors to 1 and then rescale them

K_Maldacena_FCT1_Mstar1 =  1.
K_Maldacena_HC_Mstar1   =  1.

In [45]:
# Integrands of |F_Bhat|^2 for the monopole case

special_integral = np.vectorize(special_integral)

def integrand_Maldacena_FCT1_Mstar1(LogMass, Logk):

    k = np.float_power(10., Logk)
    M = np.float_power(10., LogMass)
    dndM = Mfunction_FCT1_Mstar1(LogMass) #Jacobian already included for integration
    B_Maldacena = K_Maldacena_FCT1_Mstar1 * M**(1./2.)
    arg_Montec = 1.6 / (k * d_FCT1_Mstar1(LogMass))**3.
    I_Mald_Montecarlo = (4. * np.pi / k) * special_integral(2., arg_Montec)

    return dndM * (B_Maldacena / (a**2.)) * I_Mald_Montecarlo

def integrand_Maldacena_HC_Mstar1(LogMass, Logk):

    k = np.float_power(10., Logk)
    M = np.float_power(10., LogMass)
    dndM = Mfunction_HC_Mstar1(LogMass) #Jacobian already included for integration
    B_Maldacena = K_Maldacena_HC_Mstar1 * M**(1./2.)
    arg_Montec = 1.6 / (k * d_HC_Mstar1(LogMass))**3.
    I_Mald_Montecarlo = (4. * np.pi / k) * special_integral(2., arg_Montec)

    return dndM * (B_Maldacena / (a**2.)) * I_Mald_Montecarlo

# F_Bhat Maldacena

def F_B_hat_Mald_FCT1_Mstar1(w):
    return quadrature(integrand_Maldacena_FCT1_Mstar1, ini_LogMass, final_LogMass_FCT1_Mstar1, args=(w), tol=10000, rtol=10000, maxiter=500)[0]

def F_B_hat_Mald_HC_Mstar1(w):
    return quadrature(integrand_Maldacena_HC_Mstar1, ini_LogMass, final_LogMass_HC_Mstar1, args=(w), tol=10000, rtol=10000, maxiter=500)[0]

#Vectorize them 

function_F_B_hat_Mald_FCT1_Mstar1 = np.vectorize(F_B_hat_Mald_FCT1_Mstar1)
function_F_B_hat_Mald_HC_Mstar1   = np.vectorize(F_B_hat_Mald_HC_Mstar1)

SqFBhat_Mald_FCT1_Mstar1 = [(F_B_hat_Mald_FCT1_Mstar1(a))**2. for a in LogK_Int_FCT1_Mstar1]
SqFBhat_Mald_HC_Mstar1   = [(F_B_hat_Mald_HC_Mstar1(a))**2. for a in LogK_Int_HC_Mstar1]

# 'Normalized' version

MagSp_Mald_FCT1_Mstar1 = []
MagSp_Mald_HC_Mstar1   = []

for i in range(k_bins):
    MagSp_Mald_FCT1_Mstar1.append((10.**(3.*LogK_Int_FCT1_Mstar1[i]))/(2.*np.pi**2.)*SqFBhat_Mald_FCT1_Mstar1[i]*Psum_FCT1_Mstar1[i])
    MagSp_Mald_HC_Mstar1.append((10.**(3.*LogK_Int_HC_Mstar1[i]))/(2.*np.pi**2.)*SqFBhat_Mald_HC_Mstar1[i]*Psum_HC_Mstar1[i])

In [ ]:
SMALL_SIZE = 8
MEDIUM_SIZE = 10
BIGGER_SIZE = 12

plt.rc('font', size=15)          # controls default text sizes
plt.rc('axes', titlesize=15)     # fontsize of the axes title
plt.rc('axes', labelsize=12)    # fontsize of the x and y labels
plt.rc('xtick', labelsize=12)    # fontsize of the tick labels
plt.rc('ytick', labelsize=12)    # fontsize of the tick labels
plt.rc('legend', fontsize=9)    # legend fontsize
plt.rc('figure', titlesize=BIGGER_SIZE)  # fontsize of the figure title
fig = plt.figure(figsize=(8, 5), dpi=400)
ax = fig.add_subplot(111)
plt.semilogy(LogK_Int_FCT1_Mstar1, MagSp_Mald_FCT1_Mstar1, color='r', linestyle='-', label=r'FCT, $n_b = 1.5$, $M_* = 1.39\times 10^{2} M_\odot$')
plt.semilogy(LogK_Int_HC_Mstar1, MagSp_Mald_HC_Mstar1, color='b', linestyle='-', label=r'HC, $n_b = 3.0$, $M_* = 1.39\times 10^{2} M_\odot$')
ax.set_xlabel(r'$\log_{10}{\left(k/h/Mpc\right)}$')
ax.set_ylabel(r'$k^3\,P_B(k)/Mpc^3/h^3$')
plt.grid()
ax.legend()
#plt.savefig('Maldacena-PowerSpectrum.png')
plt.show()
plt.close()

In [ ]:
product_Mald_FCT1_Mstar1 = []
product_Mald_HC_Mstar1   = []

for i in range(k_bins):
    product_Mald_FCT1_Mstar1.append(WF_FCT1_Mstar1[i]*SqFBhat_Mald_FCT1_Mstar1[i]*Psum_FCT1_Mstar1[i]*L_jacob_FCT1_Mstar1[i])
    product_Mald_HC_Mstar1.append(WF_HC_Mstar1[i]*SqFBhat_Mald_HC_Mstar1[i]*Psum_HC_Mstar1[i]*L_jacob_HC_Mstar1[i])
    

int_product_Mald_FCT1_Mstar1 = integrate.simps(product_Mald_FCT1_Mstar1, LogK_Int_FCT1_Mstar1)
int_product_Mald_HC_Mstar1   = integrate.simps(product_Mald_HC_Mstar1, LogK_Int_HC_Mstar1)

Average_B_Mald_FCT1_Mstar1 = (int_product_Mald_FCT1_Mstar1)**(1./2.)
Average_B_Mald_HC_Mstar1   = (int_product_Mald_HC_Mstar1)**(1./2.)

print("-----------------------------------------------------")
print("B_Mald_FCT1_Mstar1 con K_mald unidad = ", Average_B_Mald_FCT1_Mstar1,"Gauss")
print("B_Mald_HC_Mstar1 con K_mald unidad   = ", Average_B_Mald_HC_Mstar1,"Gauss")

In [ ]:
#Compute the values of K such that B ~ 10^{-30}

K_Mald_FCT1_Mstar1 = 10.**(-30) / Average_B_Mald_FCT1_Mstar1
K_Mald_HC_Mstar1   = 10.**(-30) / Average_B_Mald_HC_Mstar1

print("-----------------------------------------------------")
print("K_Maldacena_FCT1_Mstar1 = ", K_Mald_FCT1_Mstar1)
print("K_Maldacena_HC_Mstar1   = ", K_Mald_HC_Mstar1)

- With the obtained values for $K$, for each scenario, go back to the cell in which those constants where set to 1, and plug the above estimations.
- Then, run up all the cells again, to verify the magnetic field magnitude is about the desired order.

## Part 2: Computing the monople abundance today

4. With the obtained values for the constant K at the two different scenarios, estimate the current monopole's density.
5. Is that density a reasonable number? Discuss.

## Monopoles density

Recall that:

- PBH density at the formation epoch proportional to  the critical density of the universe at PBH formation, $\rho_c$,

\begin{equation}
\rho_{\tiny{\mbox{PBH}}}(  a_{\text{form}})  =f_{\tiny{\mbox{PBH}}}\,f_{m}\,\rho_{\text{c}}\left(\left\langle a_{\text{form}}\right\rangle _{M\frac{dn}{dM}}\right)
\end{equation}
with $f_{\tiny{\mbox{PBH}}}$ the fraction of dark matter in PBHs; $f_{m}$ the fraction of energy in dark matter; $\left\langle a_{\text{form}}\right\rangle _{M\frac{dn}{dM}}$ the average scale factor at the formation time (correponding values taken from Sureda et. al., https://arxiv.org/pdf/2008.09683.pdf)

- Number density of magnetic monopoles at the PBH formation time:

\begin{equation}\label{eq:nmonop-form}
n_{\text{mon}}\left(  a_{\text{form}}\right)  =\left(  \frac{4\pi}{\mu
_{0}q_{m}}\right)  ^{2}K^{2}f_{\tiny{\mbox{PBH}}}\,f_{m}\,\rho_{\text{c}}\left(\left\langle a_{\text{form}}\right\rangle _{M\frac{dn}{dM}}\right)
\end{equation}

- Fraction $f_{m}$ is given by
\begin{equation}
f_{m}=\frac{\rho_{\tiny{\mbox{DM}}}\left(  \left\langle a_{\text{form}}\right\rangle_{M\frac{dn}{dM}}\right)}{\rho_{\text{c}}\left(\left\langle a_{\text{form}}\right\rangle_{M\frac{dn}{dM}}\right)}
\end{equation} 

- Critical density at PBH formation well approximated by

\begin{equation}
\rho_{\text{c}}\left(  \left\langle a_{\text{form}}\right\rangle _{M\frac
{dn}{dM}}\right)  \simeq\frac{\rho_{\text{rad,0}}}{\left(  \left\langle
a_{\text{form}}\right\rangle _{M\frac{dn}{dM}}\right)^{4}}
\end{equation} 

- Dark matter density at PBH formation
\begin{equation}
\rho_{\text{DM}}\left(  \left\langle a_{\text{form}}\right\rangle _{M\frac
{dn}{dM}}\right)  \simeq\frac{\rho_{\text{DM,0}}}{\left(  \left\langle
a_{\text{form}}\right\rangle _{M\frac{dn}{dM}}\right)^{3}},
\end{equation}
being $\rho_{\text{rad,0}}$ and $\rho_{\text{DM,0}}$ today's radiation and dark matter mass densities

- Number density of monopoles at $a>a_{\text{form}}$:

\begin{equation}\label{eq:nmonop-arbitrary-time}
n_{\text{mon}}\left(  a\right)  =n_{\text{mon}}\left(  a_{\text{form}}\right)
\left(\frac{a_{\text{form}}}{a}\right)^{3}.
\end{equation}

-----------------------------------------------------------------------------------------------

Complete the following cell with the obtained values for $K$ at each scenario formation!

In [ ]:
f_PBH = 1.

#Density values in gaussian units
I_rho_DM0 = 0.24 * 9.9e-30     #g cm^{-3}
I_rho_rad0 = 7.858e-31         #kg m^{-3}
I_rho_c0 = 1.06e-29            #g cm^{-3}

#Density values in our units
rho_DM0 = I_rho_DM0 * g_to_Msun * (cm_to_Mpc)**(-3.)   # Msum * Mpc^{-3}
rho_rad0 = I_rho_rad0 * kg_to_Msun * (m_to_Mpc)**(-3.) # Msum * Mpc^{-3}
rho_c0 = I_rho_c0 * g_to_Msun * (cm_to_Mpc)**(-3.)     # Msum * Mpc^{-3}

#------------------------------------------------------------------------------
# FCT: Complete with the value obtained before for K_FCT:

K_FCT1_Mstar1 = _____________________ * Gauss_to_OurUnits      # Msun^{1/2} Mpc^2 Coulomb^{-1} seg^{-1}
#------------------------------------------------------------------------------

a_form_FCT = 2.04e-26
rho_DM_FCT = rho_DM0 * (a_form_FCT)**(-3.)
rho_c_FCT = rho_rad0 * (a_form_FCT)**(-4.)
f_M_FCT = rho_DM_FCT / rho_c_FCT

n_monop_FCT1_Mstar1 = (4. * np.pi * (mu_0 * q_m)**(-1.))**2. * (K_FCT1_Mstar1)**2. * f_PBH * f_M_FCT * rho_c_FCT

#------------------------------------------------------------------------------
# HC: Complete with the value obtained before for K_HC:

K_HC_Mstar1 = ________________________ * Gauss_to_OurUnits   # Msun^{1/2} Mpc^2 Coulomb^{-1} seg^{-1}

#------------------------------------------------------------------------------
a_form_HC = 4.276267e-12
rho_DM_HC = rho_DM0 * (a_form_HC)**(-3.)
rho_c_HC = rho_rad0 * (a_form_HC)**(-4.)
f_M_HC = rho_DM_HC / rho_c_HC

n_monop_HC_Mstar1 = (4. * np.pi * (mu_0 * q_m)**(-1.))**2. * (K_HC_Mstar1)**2. * f_PBH * f_M_HC * rho_c_HC

print("-------------------------------------------------------------------")
print("n_monop_FCT1_Mstar1 = ", n_monop_FCT1_Mstar1, "monop/Mpc^3")
print("n_monop_HC_Mstar1   = ", n_monop_HC_Mstar1, "monop/Mpc^3")

## Monopoles abundance today

- Evaluate the monopole density function at $a=1$, we get $n_{\text{mon,0}}$
- Consider that magnetic monopoles have a mass $m_{\tiny{\mbox{GUT}}}\sim 10^{16}$ GeV (Reference: Georgi H., Glashow S. L., 1974, Phys. Rev. Lett., 32, 438)
- The fractional contribution of monopoles to the current energy density budget is:

\begin{equation}\label{eq:nmonop-today}
\Omega_{\text{mon,0}}=\frac{m_{\tiny{\mbox{GUT}}}\,n_{\text{mon,0}}}{\rho_{\text{c,0}}
}
\end{equation}

where $\rho_{\text{c,0}}$ is the critical density at $z=0$

In [ ]:
#m_GUT = 1e16 GeV

I_m_GUT = 1e16 * 1.60218e-19 #kg * m^2 * s^{-2}
#eq_m_GUT = I_m_GUT * (I_c)**(-2.) #kg

eq_m_GUT = 1e16 * 1.783e-27 #kg

m_GUT = eq_m_GUT * kg_to_Msun

n_monop0_FCT1_Mstar1 = n_monop_FCT1_Mstar1 * (a_form_FCT)**3.
n_monop0_HC_Mstar1   = n_monop_HC_Mstar1 * (a_form_HC)**3.

Omega_monop_FCT1_Mstar1 = m_GUT * n_monop0_FCT1_Mstar1 * (rho_c0)**(-1.)
Omega_monop_HC_Mstar1   = m_GUT * n_monop0_HC_Mstar1 * (rho_c0)**(-1.)

print("-----------------------------------------------------------")
print("Omega_monop_FCT1_Mstar1 = ", Omega_monop_FCT1_Mstar1)
print("Omega_monop_HC_Mstar1   = ", Omega_monop_HC_Mstar1)